In [1]:
import os
import glob
import csv
import spotipy
import spotipy.util as util
from tqdm import tqdm

def append_dict_to_csv(filename, data_dict):
    file_exists = os.path.isfile(filename)
    fieldnames = data_dict.keys()

    with open(filename, mode='a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        writer.writerow(data_dict)
        
def get_tracks(scope, client_id, client_secret, redirect_uri, user_id):
    for file in glob.glob(".cache*"):
        os.remove(file) 
    token = util.prompt_for_user_token(
                                   scope=scope,
                                   client_id=client_id,
                                   client_secret=client_secret,
                                   redirect_uri=redirect_uri)
    
#     print(token)
    if token:
        sp = spotipy.Spotify(auth=token)

        track_data = []
        offset = 0
        limit = 50

        filename = f'spotify_tracks_{user_id}.csv'
        if os.path.exists(filename):
            os.remove(filename)

        while True:
            results = sp.current_user_saved_tracks(limit=limit, offset=offset)
            total_tracks = results['total']
            if not results['items']:
                break
            
            for item in tqdm(results['items'], total = total_tracks, initial = offset, desc = "Fetching tracks"):
                try:
                    track = item['track']
                    album = sp.album(track['album']['id'])
                    artist = sp.artist(track['artists'][0]['id'])
                    audio_features = sp.audio_features([track['id']])[0]

                    metadata = {
                        'user_id':user_id,
                        'id': track['id'],
                        'name': track['name'],
                        'artist': track['artists'][0]['name'],
                        'album': track['album']['name'],
                        'release_date': track['album']['release_date'],
                        'popularity': track['popularity'],
                        'duration_ms': track['duration_ms'],
                        'explicit': track['explicit'],
                        'uri': track['uri'],
                        'album_genre': album['genres'][0] if album['genres'] else 'Unknown',
                        'artist_genre': artist['genres'][0] if artist['genres'] else 'Unknown'
                    }

                    metadata.update(audio_features)
                    track_data.append(metadata)

                    append_dict_to_csv(filename, metadata)
                except:
                    pass

            offset += limit

        return track_data
    else:
        return ("Can't get token for", username)

# username = '224hap4se67crjr4hva6nz74y'
scope = 'user-library-read'
client_id='7866736c94034da2aaec88deaa8a46aa'
client_secret='91857a08e34b457b8ff6d72f3c967d06'
redirect_uri='http://localhost/'
user_id = 'brandon'
tracks = get_tracks(scope, client_id, client_secret, redirect_uri, user_id)
print("Your Spotify tracks have been saved to 'spotify_tracks.csv'")

Using `localhost` as redirect URI without a port. Specify a port (e.g. `localhost:8080`) to allow automatic retrieval of authentication code instead of having to copy and paste the URL your browser is redirected to.


Enter the URL you were redirected to: http://localhost/?code=AQAd7eKJxZ2WIlJvkpYc2vzQXbGOSO5Msn5IxVGGIiuaglFYnT2JUlqPTG0TP1hFnW2MzQVGujezWR6NoYxrP2zKBkp3Wsi4uiTFoma7RMUJi_S3iwzximI8mOYY8KycExrvZ_xWiAEDwzco49_soX7va2WCu1JQWyCcxO7FafoYeVCW29t6Dg


Fetching tracks: 100%|██████████████████████████████████████████████████████████████████| 26/26 [00:07<00:00,  3.36it/s]

Your Spotify tracks have been saved to 'spotify_tracks.csv'
